In [1]:
#Requests for http
import requests
#Pandas for working with dataframes
import pandas
#Folium to maps
import folium
#Clusters
from sklearn.cluster import KMeans
#Colors for maps
from matplotlib import colors
#Mathematical
import math

In [2]:
#Foursquare API. The geo coordinates is our main work place
#The categoryId=4bf58dd8d48988d18194173 is for museums
url="https://api.foursquare.com/v2/venues/search?client_id=5QIWFD2A2YE30B2FLQ2Z30LSBKM3R5CJUMK0XEUZDGIMGB4W&client_secret=4NXKPDEK3ZXW0F353TJQ3YRE2HEXDXULJV3JQ5YWYSMK5ER1&ll=40.415,-3.684&v=20200801&categoryId=4bf58dd8d48988d181941735&radius=2000"

In [3]:
#Response in json
results=requests.get(url).json()

In [4]:
venues=results["response"]["venues"]
#Normalize JSON data into a flat table
dataframe=pandas.json_normalize(venues)

In [5]:
#Lots of data are not necessary
dataframe.drop(columns=["id","categories","referralId","hasPerk","location.labeledLatLngs","location.cc","location.crossStreet","venuePage.id","location.neighborhood","location.state","location.country"],inplace=True)

In [6]:
#Sort by distance
dataframe=dataframe.sort_values("location.distance")

In [7]:
#Reseting indexes
dataframe.reset_index(drop=True, inplace=True)

In [8]:
#A first idea about data
dataframe.head()

,name,location.address,location.lat,location.lng,location.distance,location.postalCode,location.city,location.formattedAddress
0,Palacio de Velázquez,"P. Duque de Fernán Núñez, s/n",40.415139,-3.681975,172,28014,Madrid,"[P. Duque de Fernán Núñez, s/n, 28014 Madrid M..."
1,Casón del Buen Retiro,"C. de Alfonso XII, 28",40.415228,-3.689057,429,28014,Madrid,"[C. de Alfonso XII, 28, 28014 Madrid Madrid, E..."
2,Museo Nacional del Prado,"C. de Ruiz de Alarcón, 23",40.413978,-3.692189,703,28014,Madrid,"[C. de Ruiz de Alarcón, 23 (P. del Prado), 280..."
3,Palacio de Linares - Casa de América,"P. de Recoletos, 2",40.420254,-3.691827,884,28014,Madrid,"[P. de Recoletos, 2 (Pl. de la Cibeles), 28014..."
4,CaixaForum Madrid,"P. del Prado, 36",40.411013,-3.693177,895,28014,Madrid,"[P. del Prado, 36, 28014 Madrid Madrid, España]"


In [9]:
#Some of venues are temporary exhibitions or not really museums
dataframe.drop([0,9,10,13,20,21,27],inplace=True)

In [10]:
#Reseting indexes again
dataframe.reset_index(drop=True, inplace=True)

In [11]:
#We need to add data about number of visitors of each museum
#Some of the data can be downloaded by http://www-2.munimadrid.es/CSE6/control/seleccionDatos?numSerie=2010300010 and seen in Sub-Directorate General of State Museums
#Data are from 2017 or 2018
#Some museums do not have easily accessible public data (in general, they are museums with few visitors in comparison)
#Data in thousands (a year)
dataframe["visitors"]=[0,2893,150,100,719,96,33,1000,0,510,256,3942,0,0,151,0,55,135,48,216,0,66]

In [12]:
#Names of indexes for "visitors" < 50
#We are not interested in museums with less than 50,000 visitors per year
index_visit=dataframe[dataframe["visitors"]<50].index
#Delete these row indexes
dataframe.drop(index_visit,inplace=True)

In [13]:
#Reseting indexes anymore
dataframe.reset_index(drop=True, inplace=True)

In [14]:
#First map
map_madrid=folium.Map(location=[40.415,-3.684],zoom_start=14)

In [15]:
for lat,lng,name,visitors in zip(dataframe["location.lat"], dataframe["location.lng"], dataframe["name"],dataframe["visitors"]):
    label = "{},{}".format(name,visitors)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
map_madrid

In [16]:
#We can review our current dataframe
dataframe

,name,location.address,location.lat,location.lng,location.distance,location.postalCode,location.city,location.formattedAddress,visitors
0,Museo Nacional del Prado,"C. de Ruiz de Alarcón, 23",40.413978,-3.692189,703,28014,Madrid,"[C. de Ruiz de Alarcón, 23 (P. del Prado), 280...",2893
1,Palacio de Linares - Casa de América,"P. de Recoletos, 2",40.420254,-3.691827,884,28014,Madrid,"[P. de Recoletos, 2 (Pl. de la Cibeles), 28014...",150
2,CaixaForum Madrid,"P. del Prado, 36",40.411013,-3.693177,895,28014,Madrid,"[P. del Prado, 36, 28014 Madrid Madrid, España]",100
3,Museo Nacional de Antropología,"C. de Alfonso XII, 68",40.407529,-3.688876,928,28014,Madrid,"[C. de Alfonso XII, 68, 28014 Madrid Madrid, E...",719
4,Panteón de Hombres Ilustres,"Calle Julian Gayarre, 3",40.406641,-3.683601,931,28014,Madrid,"[Calle Julian Gayarre, 3, 28014 Madrid Madrid,...",96
5,Exposición Rembrandt,NaN,40.416503,-3.694979,945,28014,Madrid,"[28014 Madrid Madrid, España]",1000
6,Museo Nacional Centro de Arte Reina Sofía (MNC...,"C. de Santa Isabel, 52",40.408737,-3.693822,1085,28012,Madrid,"[C. de Santa Isabel, 52, 28012 Madrid Madrid, ...",510
7,Círculo de Bellas Artes,"C. Alcalá, 42",40.418486,-3.696612,1137,28014,Madrid,"[C. Alcalá, 42, 28014 Madrid Madrid, España]",256
8,La Neomudéjar,Calle Antonio Nebrija,40.404717,-3.683918,1144,NaN,Madrid,"[Calle Antonio Nebrija, Madrid Madrid, España]",3942
9,Sala de Exposiciones Alcalá 31,"C/ Alcalá, 31",40.418295,-3.698258,1262,28014,Madrid,"[C/ Alcalá, 31, 28014 Madrid Madrid, España]",151


In [17]:
#To prepare the partitioning of the data space (we are interested in distance to our main worksite and in number of visitors to sort out)
df_clusters=pandas.DataFrame(data=dataframe,columns=["location.distance","visitors"])

In [18]:
#We are going to set up three offices
num_clusters=3

In [19]:
kmeans=KMeans(n_clusters=num_clusters,random_state=0).fit(df_clusters)

In [20]:
#Labels to our dataframe
dataframe.insert(0,"cluster.labels",kmeans.labels_)

In [21]:
#We can simply set 4 colours for markers
colors=["#e50000","#21fc0d","#fffe40","#ffff"]

In [22]:
#Our new map (clusters)
map_madrid_cl=folium.Map(location=[40.415,-3.684],zoom_start=14)

In [23]:
for lat,lng,name,visitors,cluster in zip(dataframe["location.lat"],dataframe["location.lng"],dataframe["name"],dataframe["visitors"],dataframe["cluster.labels"]):
    label = "{}, visitors:{}".format(name,visitors)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color="#000000",
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.9,
        parse_html=False).add_to(map_madrid_cl)  
map_madrid_cl

In [24]:
#We want to create a dataframe with the closest museum of each cluster
df0=dataframe[dataframe["cluster.labels"]==0]
dfsub0=pandas.DataFrame(df0[df0["location.distance"]==min(df0["location.distance"])])

In [25]:
df1=dataframe[dataframe["cluster.labels"]==1]
dfsub1=pandas.DataFrame(df1[df1["location.distance"]==min(df1["location.distance"])])

In [26]:
df2=dataframe[dataframe["cluster.labels"]==2]
dfsub2=pandas.DataFrame(df2[df2["location.distance"]==min(df2["location.distance"])])

In [27]:
df_base=pandas.concat([dfsub0,dfsub1,dfsub2])

In [28]:
df_base

,cluster.labels,name,location.address,location.lat,location.lng,location.distance,location.postalCode,location.city,location.formattedAddress,visitors
10,0,Espacio Fundación Telefónica,"C. Fuencarral, 3",40.420392,-3.701617,1609,28004,Madrid,"[C. Fuencarral, 3, 28004 Madrid Madrid, España]",55
0,1,Museo Nacional del Prado,"C. de Ruiz de Alarcón, 23",40.413978,-3.692189,703,28014,Madrid,"[C. de Ruiz de Alarcón, 23 (P. del Prado), 280...",2893
1,2,Palacio de Linares - Casa de América,"P. de Recoletos, 2",40.420254,-3.691827,884,28014,Madrid,"[P. de Recoletos, 2 (Pl. de la Cibeles), 28014...",150


In [29]:
#Adding the main Retiro Park workplace
df_base=df_base.append({"name":"MAIN BASE","location.lat":40.415,"location.lng":-3.684},ignore_index=True)

In [30]:
df_base

,cluster.labels,name,location.address,location.lat,location.lng,location.distance,location.postalCode,location.city,location.formattedAddress,visitors
0,0.0,Espacio Fundación Telefónica,"C. Fuencarral, 3",40.420392,-3.701617,1609.0,28004,Madrid,"[C. Fuencarral, 3, 28004 Madrid Madrid, España]",55.0
1,1.0,Museo Nacional del Prado,"C. de Ruiz de Alarcón, 23",40.413978,-3.692189,703.0,28014,Madrid,"[C. de Ruiz de Alarcón, 23 (P. del Prado), 280...",2893.0
2,2.0,Palacio de Linares - Casa de América,"P. de Recoletos, 2",40.420254,-3.691827,884.0,28014,Madrid,"[P. de Recoletos, 2 (Pl. de la Cibeles), 28014...",150.0
3,NaN,MAIN BASE,NaN,40.415000,-3.684000,NaN,NaN,NaN,NaN,NaN


In [31]:
#Now we can establish our auxiliar offices and check
i=-1
for lat,lng,name in zip(df_base["location.lat"],df_base["location.lng"],df_base["name"]):
    label = "{}".format(name)
    label = folium.Popup(label, parse_html=True)
    i=i+1
    folium.CircleMarker(
        [lat, lng],
        radius=11,
        popup=label,
        color="#0000ff",
        fill=True,
        fill_color=colors[i],
        fill_opacity=0.9,
        parse_html=False).add_to(map_madrid_cl)  
map_madrid_cl

In [32]:
#Our dataframe for each 
df0

,cluster.labels,name,location.address,location.lat,location.lng,location.distance,location.postalCode,location.city,location.formattedAddress,visitors
10,0,Espacio Fundación Telefónica,"C. Fuencarral, 3",40.420392,-3.701617,1609,28004,Madrid,"[C. Fuencarral, 3, 28004 Madrid Madrid, España]",55
11,0,Museum of Illusions Madrid,C. Doctor Cortezo 8,40.413364,-3.703821,1689,NaN,NaN,"[C. Doctor Cortezo 8, España]",135
12,0,Museo de Historia (Museo Municipal de Madrid),"C. Fuencarral, 78",40.425690,-3.700972,1866,28004,Madrid,"[C. Fuencarral, 78, 28004 Madrid Madrid, España]",216
13,0,Museo metro,NaN,40.431658,-3.700583,2326,28010,Madrid,"[28010 Madrid Madrid, España]",66


In [33]:
df1

,cluster.labels,name,location.address,location.lat,location.lng,location.distance,location.postalCode,location.city,location.formattedAddress,visitors
0,1,Museo Nacional del Prado,"C. de Ruiz de Alarcón, 23",40.413978,-3.692189,703,28014,Madrid,"[C. de Ruiz de Alarcón, 23 (P. del Prado), 280...",2893
8,1,La Neomudéjar,Calle Antonio Nebrija,40.404717,-3.683918,1144,NaN,Madrid,"[Calle Antonio Nebrija, Madrid Madrid, España]",3942


In [34]:
df2

,cluster.labels,name,location.address,location.lat,location.lng,location.distance,location.postalCode,location.city,location.formattedAddress,visitors
1,2,Palacio de Linares - Casa de América,"P. de Recoletos, 2",40.420254,-3.691827,884,28014,Madrid,"[P. de Recoletos, 2 (Pl. de la Cibeles), 28014...",150
2,2,CaixaForum Madrid,"P. del Prado, 36",40.411013,-3.693177,895,28014,Madrid,"[P. del Prado, 36, 28014 Madrid Madrid, España]",100
3,2,Museo Nacional de Antropología,"C. de Alfonso XII, 68",40.407529,-3.688876,928,28014,Madrid,"[C. de Alfonso XII, 68, 28014 Madrid Madrid, E...",719
4,2,Panteón de Hombres Ilustres,"Calle Julian Gayarre, 3",40.406641,-3.683601,931,28014,Madrid,"[Calle Julian Gayarre, 3, 28014 Madrid Madrid,...",96
5,2,Exposición Rembrandt,NaN,40.416503,-3.694979,945,28014,Madrid,"[28014 Madrid Madrid, España]",1000
6,2,Museo Nacional Centro de Arte Reina Sofía (MNC...,"C. de Santa Isabel, 52",40.408737,-3.693822,1085,28012,Madrid,"[C. de Santa Isabel, 52, 28012 Madrid Madrid, ...",510
7,2,Círculo de Bellas Artes,"C. Alcalá, 42",40.418486,-3.696612,1137,28014,Madrid,"[C. Alcalá, 42, 28014 Madrid Madrid, España]",256
9,2,Sala de Exposiciones Alcalá 31,"C/ Alcalá, 31",40.418295,-3.698258,1262,28014,Madrid,"[C/ Alcalá, 31, 28014 Madrid Madrid, España]",151


In [35]:
#We are going to research of visitors per year in each cluster
print("Cluster of Real Academia de Bellas Artes de San Fernando office:")
print(df0.visitors.sum())
print("Cluster of Museo Nacional del Prado:")
print(df1.visitors.sum())
print("Cluster of Museo Naval de Madrid:")
print(df2.visitors.sum())

Cluster of Real Academia de Bellas Artes de San Fernando office:
472
Cluster of Museo Nacional del Prado:
6835
Cluster of Museo Naval de Madrid:
2982


In [36]:
#Our idea is to assign at least as many educators as museums in a cluster and one more for every million visitors a year
#We are going to assign a coordinator in Main Base
df0_educators=math.floor(len(df0.index)+df0.visitors.sum()/1000)
df1_educators=math.floor(len(df1.index)+df1.visitors.sum()/1000)
df2_educators=math.floor(len(df2.index)+df2.visitors.sum()/1000)

In [37]:
#The last dataframe
df_final=df_base
df_final.drop(columns=["cluster.labels","location.postalCode","location.city","location.formattedAddress"],inplace=True)
df_final=df_final.assign(staff=[df0_educators,df1_educators,df2_educators,1])
df_final

,name,location.address,location.lat,location.lng,location.distance,visitors,staff
0,Espacio Fundación Telefónica,"C. Fuencarral, 3",40.420392,-3.701617,1609.0,55.0,4
1,Museo Nacional del Prado,"C. de Ruiz de Alarcón, 23",40.413978,-3.692189,703.0,2893.0,8
2,Palacio de Linares - Casa de América,"P. de Recoletos, 2",40.420254,-3.691827,884.0,150.0,10
3,MAIN BASE,NaN,40.415000,-3.684000,NaN,NaN,1


In [38]:
#Finally we can see in the map the location and the staff of each office and the museums assigned to each one
i=-1
for lat,lng,name,visitors,staff in zip(df_base["location.lat"],df_base["location.lng"],df_base["name"],df_final["visitors"],df_final["staff"]):
    label = "{}, visitors: {}, staff: {}".format(name,visitors,staff)
    label = folium.Popup(label, parse_html=True)
    i=i+1
    folium.CircleMarker(
        [lat, lng],
        radius=11,
        popup=label,
        color="#0000ff",
        fill=True,
        fill_color=colors[i],
        fill_opacity=0.9,
        parse_html=False).add_to(map_madrid_cl)
map_madrid_cl